In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import pandas as pd
import numpy as np
import json
import openweathermapy.core as owm
from datetime import datetime
from citipy import citipy
from pprint import pprint
from api_keys import api_key

In [ ]:
# Create empty data frame and add columns for city, country, latitude and longitude
city_df = pd.DataFrame()
city_df["City"] = ""
city_df["Country"] = ""
# Randomly generate numbers for latitude and longitude and loop through 500 times
city_df["Latitude"] = [np.random.uniform(-90, 90) for x in range(1500)]
city_df["Longitude"] = [np.random.uniform(-180, 180) for x in range(1500)]
# Use latitude and longitude to find city and country by using interrows and citipy
for index, row in city_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_df.set_value(index, "City", citipy.nearest_city(lat, lng).city_name)
    city_df.set_value(index, "Country", citipy.nearest_city(lat, lng).country_code)
# Delete any possible duplicate cities and reset index
city_df = city_df.drop_duplicates(["City", "Country"])
city_df = city_df.reset_index()
# Update data frame to just city and country column
city_df = city_df[["City", "Country"]]
city_df = city_df.iloc[:500]
city_df

In [ ]:
# Perform API calls and display results in Data Frame
city_df["Cloudiness"] = ""
city_df["Date"] = ""
city_df["Humidity"] = ""
city_df["Latitude"] = ""
city_df["Longitude"] = ""
city_df["Max Temperature"] = ""
city_df["Wind Speed"] = ""

# Create row counter and set it to one
row_counter = 1
# Create a dictionary with parameters
settings = {"appid": api_key, "units": "imperial"}
# Create a base url
base_url = "http://api.openweathermap.org/data/2.5/weather?"
# Create query url
query_url = f"{base_url}appid={settings['appid']}&units={settings['units']}&q="

print("Initiating Data Retrieval...Please Standy By")
print("---------------------------")
# Iterrows in to loop city row in query url
for index, row in city_df.iterrows():
    try:
        city_response = requests.get(query_url + row["City"] + "," + row["Country"])
        print(f"Processing Record {row_counter} | {row['City']}, {row['Country']}")
        print(city_response.url)
        # Update row counter
        row_counter += 1
    
        # Turn cities response into json with owm module
        city_response = owm.get_current(row["City"], **settings)
    
        # Create summary variables to retrieve certain data from cities response
        lat_summary = ["coord.lat"]
        lon_summary = ["coord.lon"]
        temp_summary = ["main.temp_max"]
        humidity_summary = ["main.humidity"]
        cloud_summary = ["clouds.all"]
        wind_summary = ["wind.speed"]
        date_summary = ["dt"]
    
        # Add retrieved data into cities_list data frame
        city_df.set_value(index, "Cloudiness", city_response (*cloud_summary))
        city_df.set_value(index, "Date", city_response (*date_summary))
        city_df.set_value(index, "Humidity", city_response(*humidity_summary))
        city_df.set_value(index, "Latitude", city_response(*lat_summary))
        city_df.set_value(index, "Longitude", city_response(*lon_summary))
        city_df.set_value(index, "Max Temperature", city_response(*temp_summary))
        city_df.set_value(index, "Wind Speed", city_response(*wind_summary))
    except:
        print(f"**Weather data does not exist for {row['City']}, {row['Country']}**")
        continue
    
print("---------------------------")
print("Data Retrieval Complete. There is a GOD!!!")
print("---------------------------")

Initiating Data Retrieval...Please Standy By
---------------------------
Processing Record 1 | binga, cd
http://api.openweathermap.org/data/2.5/weather?appid=YOUR%20KEY%20HERE!&units=imperial&q=binga,cd
**Weather data does not exist for binga, cd**
Processing Record 2 | ushuaia, ar
http://api.openweathermap.org/data/2.5/weather?appid=YOUR%20KEY%20HERE!&units=imperial&q=ushuaia,ar
**Weather data does not exist for ushuaia, ar**
Processing Record 3 | hanting, cn
http://api.openweathermap.org/data/2.5/weather?appid=YOUR%20KEY%20HERE!&units=imperial&q=hanting,cn
**Weather data does not exist for hanting, cn**
Processing Record 4 | zhanatas, kz
http://api.openweathermap.org/data/2.5/weather?appid=YOUR%20KEY%20HERE!&units=imperial&q=zhanatas,kz
**Weather data does not exist for zhanatas, kz**
Processing Record 5 | aklavik, ca
http://api.openweathermap.org/data/2.5/weather?appid=YOUR%20KEY%20HERE!&units=imperial&q=aklavik,ca
**Weather data does not exist for aklavik, ca**
Processing Record 6 

**Weather data does not exist for ust-ordynskiy, ru**
Processing Record 46 | guerrero negro, mx
http://api.openweathermap.org/data/2.5/weather?appid=YOUR%20KEY%20HERE!&units=imperial&q=guerrero%20negro,mx
**Weather data does not exist for guerrero negro, mx**
Processing Record 47 | svetlyy, ru
http://api.openweathermap.org/data/2.5/weather?appid=YOUR%20KEY%20HERE!&units=imperial&q=svetlyy,ru
**Weather data does not exist for svetlyy, ru**
Processing Record 48 | hermanus, za
http://api.openweathermap.org/data/2.5/weather?appid=YOUR%20KEY%20HERE!&units=imperial&q=hermanus,za
**Weather data does not exist for hermanus, za**
Processing Record 49 | barrow, us
http://api.openweathermap.org/data/2.5/weather?appid=YOUR%20KEY%20HERE!&units=imperial&q=barrow,us
**Weather data does not exist for barrow, us**
Processing Record 50 | comodoro rivadavia, ar
http://api.openweathermap.org/data/2.5/weather?appid=YOUR%20KEY%20HERE!&units=imperial&q=comodoro%20rivadavia,ar
**Weather data does not exist fo

In [ ]:
# Temperature (F) vs. Latitude
timestamp = city_df["Date"][0]
converted_timestamp = datetime.utcfromtimestamp(timestamp)
shortened_timestamp = datetime.date(converted_timestamp)

Lat = pd.to_numeric(city_df["Latitude"])
Temp = pd.to_numeric(city_df["Max Temperature"])

with sns.axes_style("darkgrid"):
    sns.regplot(data=city_df, x=Lat, y=Temp, fit_reg=False)
plt.title(f"City Latitude vs Max Temperature ({shortened_timestamp})")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.scatter(x=Lat, y=Temp, color="blue", alpha=1, edgecolor="black")
plt.show()

In [ ]:
# Humidity (%) vs. Latitude
Hum = pd.to_numeric(city_df["Humidity"])

with sns.axes_style("darkgrid"):
    sns.regplot(data=city_df, x=Lat, y=Hum, fit_reg=False)
plt.title(f"City Latitude vs Humidity ({shortened_timestamp})")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.scatter(x=Lat, y=Hum, color="blue", alpha=1, edgecolor="black")
plt.show()

In [ ]:
# Cloudiness (%) vs. Latitude
Cloud = pd.to_numeric(city_df["Cloudiness"])

with sns.axes_style("darkgrid"):
    sns.regplot(data=city_df, x=Lat, y=Cloud, fit_reg=False)
plt.title(f"City Latitude vs Cloudiness ({shortened_timestamp})")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.scatter(x=Lat, y=Cloud, color="blue", alpha=1, edgecolor="black")
plt.show()

In [ ]:
# Wind Speed (mph) vs. Latitude
Wind = pd.to_numeric(city_df["Wind Speed"])

with sns.axes_style("darkgrid"):
    sns.regplot(data=city_df, x=Lat, y=Wind, fit_reg=False)
plt.title(f"City Latitude vs Wind Speed ({shortened_timestamp})")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.scatter(x=Lat, y=Wind, color="blue", alpha=1, edgecolor="black")
plt.show()
